# Alonhadat Data Preprocessing
This notebook contains the steps to preprocess the alonhadat.csv dataset.
The steps are derived from the exploration phase and focus on cleaning and transforming the data for modeling.

## 1. Imports
Import necessary libraries for data manipulation and regular expressions.

In [38]:
import pandas as pd
import numpy as np
import re
# from datetime import datetime # Import if date processing is added later
# import warnings
# warnings.filterwarnings('ignore') # Uncomment if needed

## 2. Load Data
Load the raw dataset.

In [39]:
df = pd.read_csv('../Data Collection/Datasets/batdongsan.com/raw/batdongsan.csv')
print("Original DataFrame shape:", df.shape)
df.head()

Original DataFrame shape: (17702, 18)


,Diện tích,Hướng ban công,Hướng nhà,Mặt tiền,Mức giá,Mức giá internet,Mức giá nước,Mức giá điện,Nội thất,Pháp lý,Số phòng ngủ,"Số phòng tắm, vệ sinh",Số tầng,Thời gian dự kiến vào ở,Tiện ích,title,url,Đường vào
0,107 m²,NaN,NaN,NaN,Thỏa thuận,NaN,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,3 phòng,2 phòng,NaN,NaN,NaN,"Bán căn hộ Han Jardin T6 và T7 view hồ, giá từ...",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,NaN
1,65 m²,NaN,NaN,NaN,"3,4 tỷ",NaN,NaN,NaN,Đầy đủ,Sổ đỏ/ Sổ hồng,2 phòng,2 phòng,NaN,NaN,NaN,Gia đình em bán căn hộ tòa Park - 65m2 - full ...,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,NaN
2,50 m²,NaN,Bắc,4 m,"6,55 tỷ",NaN,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,NaN,NaN,NaN,NaN,NaN,"Bán đất dịch vụ 3 phường Đồng Mai, quận Hà Đôn...",https://batdongsan.com.vn/ban-dat-duong-co-ban...,13 m
3,150 m²,NaN,Đông - Nam,5 m,"7,99 tỷ",NaN,NaN,NaN,Đầy đủ,Sổ đỏ/ Sổ hồng,2 phòng,1 phòng,1 tầng,NaN,NaN,Giảm 500 Triệu Nhà C4 5mx30m Mặt Tiền Đường Ph...,https://batdongsan.com.vn/ban-nha-mat-pho-duon...,20 m
4,62 m²,NaN,NaN,NaN,"1,99 tỷ",NaN,NaN,NaN,NaN,Hợp đồng mua bán,2 phòng,2 phòng,NaN,NaN,NaN,GIA ĐÌNH CÓ 2 CĂN 62M2 HƯỚNG ĐÔNG NAM CẦN BÁN....,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,NaN


## 3. Initial Cleaning and Column Selection
Select relevant columns for the analysis and remove duplicate rows.

In [40]:
# Choosing relevant columns
df = df[['Diện tích', 'Hướng ban công', 'Hướng nhà', 'Mặt tiền', 'Mức giá', 'Nội thất', 'Pháp lý', 'Số phòng ngủ', 'Số phòng tắm, vệ sinh', 'Số tầng', 'title', 'Đường vào']]
print("Shape after column selection:", df.shape)

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

if duplicates > 0:
    df = df.drop_duplicates()
    print(f"Duplicates removed. New shape: {df.shape}")
df.head()

Shape after column selection: (17702, 12)
Number of duplicate rows: 7271
Duplicates removed. New shape: (10431, 12)


,Diện tích,Hướng ban công,Hướng nhà,Mặt tiền,Mức giá,Nội thất,Pháp lý,Số phòng ngủ,"Số phòng tắm, vệ sinh",Số tầng,title,Đường vào
0,107 m²,NaN,NaN,NaN,Thỏa thuận,NaN,Sổ đỏ/ Sổ hồng,3 phòng,2 phòng,NaN,"Bán căn hộ Han Jardin T6 và T7 view hồ, giá từ...",NaN
1,65 m²,NaN,NaN,NaN,"3,4 tỷ",Đầy đủ,Sổ đỏ/ Sổ hồng,2 phòng,2 phòng,NaN,Gia đình em bán căn hộ tòa Park - 65m2 - full ...,NaN
2,50 m²,NaN,Bắc,4 m,"6,55 tỷ",NaN,Sổ đỏ/ Sổ hồng,NaN,NaN,NaN,"Bán đất dịch vụ 3 phường Đồng Mai, quận Hà Đôn...",13 m
3,150 m²,NaN,Đông - Nam,5 m,"7,99 tỷ",Đầy đủ,Sổ đỏ/ Sổ hồng,2 phòng,1 phòng,1 tầng,Giảm 500 Triệu Nhà C4 5mx30m Mặt Tiền Đường Ph...,20 m
4,62 m²,NaN,NaN,NaN,"1,99 tỷ",NaN,Hợp đồng mua bán,2 phòng,2 phòng,NaN,GIA ĐÌNH CÓ 2 CĂN 62M2 HƯỚNG ĐÔNG NAM CẦN BÁN....,NaN


## 5. Numeric Feature Conversion
Convert 'area', 'bedrooms', and 'floors' columns to numeric types, handling potential errors.

In [41]:
# Convert area, bedrooms, floors, and price to numeric types

# --- Area --- 
# Extracts numbers (e.g., "80", "36.5") from strings like "80 m2"
# User request: "1.111" should mean 1111, so dots are thousand separators. Commas are decimal separators.
area_series = df['Diện tích'].astype(str)
# Remove dots (thousand separators)
area_series_no_dots = area_series.str.replace(r'\.', '', regex=True)
# Replace commas with dots (decimal separators)
area_series_comma_as_dot = area_series_no_dots.str.replace(r',', '.', regex=True)
# Extract the number part
df['Diện tích'] = pd.to_numeric(area_series_comma_as_dot.str.extract(r'(\d+(?:\.\d+)?)', expand=False), errors='coerce')
df['Diện tích'] = df['Diện tích'].fillna(0)

# --- Bedrooms ---
df['Số phòng ngủ'] = pd.to_numeric(df['Số phòng ngủ'].astype(str).str.extract(r'(\d+)', expand=False), errors='coerce').astype('Int64')
df['Số phòng ngủ'] = df['Số phòng ngủ'].fillna(0)

# --- Floors ---
df['Số tầng'] = pd.to_numeric(df['Số tầng'].astype(str).str.extract(r'(\d+)', expand=False), errors='coerce').astype('Int64')
df['Số tầng'] = df['Số tầng'].fillna(0)

# --- Toilets, bathroom ---
df['Số phòng tắm, vệ sinh'] = df['Số phòng tắm, vệ sinh'].astype(str).str.extract(r'(\d+)', expand=False)
df['Số phòng tắm, vệ sinh'] = pd.to_numeric(df['Số phòng tắm, vệ sinh'], errors='coerce').astype('Int64')
df['Số phòng tắm, vệ sinh'] = df['Số phòng tắm, vệ sinh'].fillna(0)

# --- Road width ---
df['Đường vào'] = df['Đường vào'].astype(str).str.extract(r'(\d+)', expand=False)
df['Đường vào'] = pd.to_numeric(df['Đường vào'], errors='coerce')
df['Đường vào'] = df['Đường vào'].fillna(0)

# --- Facade width ---
df['Mặt tiền'] = df['Mặt tiền'].astype(str).str.extract(r'(\d+)', expand=False)
df['Mặt tiền'] = pd.to_numeric(df['Mặt tiền'], errors='coerce')
df['Mặt tiền'] = df['Mặt tiền'].fillna(0)

# --- Balcony direction ---
# Map direction to numeric values
direction_map = {
    'Bắc': int(0),
    'Đông-Bắc': int(1),
    'Đông': int(2),
    'Đông-Nam': int(3),
    'Nam': int(4),
    'Tây-Nam': int(5),
    'Tây': int(6),
    'Tây-Bắc': int(7)
}
# Map missing values in balcony direction: 0 if missing, else 1-8 based on direction_map
df['Hướng ban công'] = df['Hướng ban công'].map(direction_map)
df['Hướng ban công'] = df['Hướng ban công'].apply(lambda x: 0 if pd.isnull(x) else x + 1)
df['Hướng ban công'] = df['Hướng ban công'].astype('Int64')

# --- House direction ---
df['Hướng nhà'] = df['Hướng nhà'].map(direction_map)
df['Hướng nhà'] = df['Hướng nhà'].apply(lambda x: 0 if pd.isnull(x) else x + 1)
df['Hướng nhà'] = df['Hướng nhà'].astype('Int64')

# --- Interior ---
# Map 'Đầy đủ' to 2, 'Cơ bản' to 1, and NaN to 0
df['Nội thất'] = df['Nội thất'].apply(lambda x: 2 if x == 'Đầy đủ' else (1 if pd.notnull(x) else 0))

# --- Legality ---
# Map 2 to 'Sổ đỏ/Sổ hồng', 1 to 'Others', and 0 to NaN
df['Pháp lý'] = df['Pháp lý'].apply(lambda x: 2 if x == 'Sổ đỏ/ Sổ hồng' else (1 if pd.notnull(x) else 0))

# Rename Vietnamese columns to English equivalents
column_rename_map = {
    'Diện tích': 'area',
    'Hướng ban công': 'balcony_direction',
    'Hướng nhà': 'house_direction',
    'Mặt tiền': 'facade_width',
    'Mức giá': 'price',
    'Nội thất': 'interior',
    'Pháp lý': 'legality',
    'Số phòng ngủ': 'bedrooms',
    'Số phòng tắm, vệ sinh': 'bathrooms',
    'Số tầng': 'floors',
    'title': 'title',
    'Đường vào': 'road_width',
}
df = df.rename(columns=column_rename_map)

print("Data types after numeric conversion:")
df.info()

print("\nDataFrame head after numeric conversion:")
df.head()


Data types after numeric conversion:
<class 'pandas.core.frame.DataFrame'>
Index: 10431 entries, 0 to 17694
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   area               10431 non-null  float64
 1   balcony_direction  10431 non-null  Int64  
 2   house_direction    10431 non-null  Int64  
 3   facade_width       10431 non-null  float64
 4   price              10430 non-null  object 
 5   interior           10431 non-null  int64  
 6   legality           10431 non-null  int64  
 7   bedrooms           10431 non-null  Int64  
 8   bathrooms          10431 non-null  Int64  
 9   floors             10431 non-null  Int64  
 10  title              10430 non-null  object 
 11  road_width         10431 non-null  float64
dtypes: Int64(5), float64(3), int64(2), object(2)
memory usage: 1.1+ MB

DataFrame head after numeric conversion:


,area,balcony_direction,house_direction,facade_width,price,interior,legality,bedrooms,bathrooms,floors,title,road_width
0,107.0,0,0,0.0,Thỏa thuận,0,2,3,2,0,"Bán căn hộ Han Jardin T6 và T7 view hồ, giá từ...",0.0
1,65.0,0,0,0.0,"3,4 tỷ",2,2,2,2,0,Gia đình em bán căn hộ tòa Park - 65m2 - full ...,0.0
2,50.0,0,1,4.0,"6,55 tỷ",0,2,0,0,0,"Bán đất dịch vụ 3 phường Đồng Mai, quận Hà Đôn...",13.0
3,150.0,0,0,5.0,"7,99 tỷ",2,2,2,1,1,Giảm 500 Triệu Nhà C4 5mx30m Mặt Tiền Đường Ph...,20.0
4,62.0,0,0,0.0,"1,99 tỷ",0,1,2,2,0,GIA ĐÌNH CÓ 2 CĂN 62M2 HƯỚNG ĐÔNG NAM CẦN BÁN....,0.0


## 6. Price Processing
Parse the 'price' column, converting string representations (e.g., "X tỷ", "Y triệu") into a standardized numeric 'price_converted' column (in millions).

In [42]:
def parse_price(price_str):
    if pd.isna(price_str):
        return np.nan

    price_str_lower = str(price_str).lower()

    if 'thỏa thuận' in price_str_lower:
        return np.nan

    cleaned_price_str = price_str_lower.replace(',', '.')
    num_part_match = re.search(r'(\d+(?:\.\d+)?)', cleaned_price_str)
    if not num_part_match:
        return np.nan

    num_val = float(num_part_match.group(1))
    if 'tháng' in price_str_lower:
        return np.nan
    if 'tỷ' in price_str_lower:
        return num_val * 1e3  # Billion to million
    elif 'triệu' in price_str_lower:
        return num_val  # Million
    elif 'triệu/m' in price_str_lower:
        return num_val * df['area']  # Million per square meter
    return num_val /1e6  # Default to million


df['price'] = df['price'].apply(parse_price)

# Remove invalid 'Mức giá' entries (null or zero)
df = df[df['price'].notnull() & (df['price'] != 0)]

print("Price column parsed and created.")
df['price'].head()
df['price'].describe()

Price column parsed and created.


count      8234.000000
mean      13283.299621
std       30969.818888
min           0.000020
25%        2850.000000
50%        5700.000000
75%       12000.000000
max      740000.000000
Name: price, dtype: float64

## 7. Outlier Removal (Initial Columns)
Define and apply the IQR method to remove outliers from 'area', 'bedrooms', 'floors', and 'price_converted'.

In [43]:
# Function to remove outliers using IQR
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Columns to remove outliers from
outlier_columns = ['area', 'bedrooms', 'floors', 'price', 'bathrooms', 'road_width', 'facade_width']

print(f"Shape before outlier removal: {df.shape}")

df_no_outliers = df.copy()
for col in outlier_columns:
    df_no_outliers = remove_outliers_iqr(df_no_outliers, col)

print(f"Shape after outlier removal: {df_no_outliers.shape}")

# Display summary statistics after outlier removal
print("\nSummary statistics after outlier removal:")
print(df_no_outliers[outlier_columns].describe())

Shape before outlier removal: (8234, 12)
Shape after outlier removal: (5613, 12)

Summary statistics after outlier removal:
              area  bedrooms    floors         price  bathrooms   road_width  \
count  5613.000000    5613.0    5613.0   5613.000000     5613.0  5613.000000   
mean     83.125208  1.902369  1.029396   5689.091504   1.619989     2.609300   
std      39.802832  1.540988  1.678665   4191.792009   1.428907     3.949932   
min       7.000000       0.0       0.0      0.000110        0.0     0.000000   
25%      57.000000       0.0       0.0   2690.000000        0.0     0.000000   
50%      75.000000       2.0       0.0   4750.000000        2.0     0.000000   
75%     100.000000       3.0       2.0   7500.000000        2.0     5.000000   
max     260.000000       7.0       7.0  19900.000000        5.0    15.000000   

       facade_width  
count   5613.000000  
mean       2.056120  
std        2.678527  
min        0.000000  
25%        0.000000  
50%        0.000000  
7

## 8. Feature Engineering: Price per m²
Create the 'price_per_m2' feature using the cleaned 'price_converted' and 'area'.

In [44]:
# Ensure 'area' is not zero or NA before division
df = df[df['area'].notna() & (df['area'] > 0)]
# Calculate price per m²
df_no_outliers['price_per_m2'] = df_no_outliers['price'] / df_no_outliers['area']

# Display the DataFrame with the new column
print("\nDataFrame with 'price_per_m2' column:")
print(df_no_outliers[['area', 'price', 'price_per_m2']].head())

print("\nSummary statistics for 'price_per_m2':")
print(df_no_outliers['price_per_m2'].describe())

df = df_no_outliers.copy()


DataFrame with 'price_per_m2' column:
    area   price  price_per_m2
1   65.0  3400.0     52.307692
2   50.0  6550.0    131.000000
4   62.0  1990.0     32.096774
6  120.0  7300.0     60.833333
7   77.1  3300.0     42.801556

Summary statistics for 'price_per_m2':
count    5.613000e+03
mean     8.130790e+01
std      6.813939e+01
min      6.818182e-07
25%      3.870968e+01
50%      6.315789e+01
75%      1.024242e+02
max      7.250000e+02
Name: price_per_m2, dtype: float64


## 9. Outlier Removal for Price per m²
Apply IQR outlier removal specifically to the newly created 'price_per_m2' column.

In [45]:
print(f"Shape before outlier removal for 'price_per_m2': {df.shape}")
if 'price_per_m2' in df.columns and df['price_per_m2'].notna().sum() > 0:
    df = remove_outliers_iqr(df, 'price_per_m2')
    print(f"Shape after outlier removal for 'price_per_m2': {df.shape}")
    print("\nSummary statistics for 'price_per_m2' after outlier removal:")
    print(df['price_per_m2'].describe())
else:
    print("Skipping outlier removal for 'price_per_m2' as it's missing or all NA.")

Shape before outlier removal for 'price_per_m2': (5613, 13)
Shape after outlier removal for 'price_per_m2': (5212, 13)

Summary statistics for 'price_per_m2' after outlier removal:
count    5.212000e+03
mean     6.752057e+01
std      4.510518e+01
min      6.818182e-07
25%      3.655128e+01
50%      5.921053e+01
75%      9.013358e+01
max      1.977778e+02
Name: price_per_m2, dtype: float64


## 10. Final Processed DataFrame
Display information about the final processed DataFrame.

In [46]:
print("Final DataFrame head:")
print(df.head())
print("\nFinal DataFrame info:")
df.info()
print("\nFinal DataFrame shape:", df.shape)

# Optional: Save the processed DataFrame
# df.to_csv('../Data Preprocessing/alonhadat_processed.csv', index=False)
# print("\nProcessed DataFrame saved to '../Data Preprocessing/alonhadat_processed.csv'")

Final DataFrame head:
    area  balcony_direction  house_direction  facade_width   price  interior  \
1   65.0                  0                0           0.0  3400.0         2   
2   50.0                  0                1           4.0  6550.0         0   
4   62.0                  0                0           0.0  1990.0         0   
6  120.0                  0                0           7.0  7300.0         1   
7   77.1                  0                0           0.0  3300.0         2   

   legality  bedrooms  bathrooms  floors  \
1         2         2          2       0   
2         2         0          0       0   
4         1         2          2       0   
6         2         7          3       3   
7         2         3          2       0   

                                               title  road_width  price_per_m2  
1  Gia đình em bán căn hộ tòa Park - 65m2 - full ...         0.0     52.307692  
2  Bán đất dịch vụ 3 phường Đồng Mai, quận Hà Đôn...        13.0    13

In [47]:
# Save the processed DataFrame
df.to_csv('../Data Preprocessing/batdongsan_processed.csv', index=False)